# Scrapper de elementos de la página del SSN

Lo primero que debemos hacer es obtener el registro de sismos para un determinado periodo. Para eso, vamos a hacer uso de la biblioteca `requests` de Python, y despues de `beautifulsoup4` para hacer el scrapping de la página que se obtiene con `requests`, y con `pandas` un DataFrame para poder escribir todo a un csv y trabajar con él más adelante

In [1]:
import requests #Para mandar mensajes POST a un URL
from bs4 import BeautifulSoup
import pandas as pd

Después de __perder un buen rato por la falta de documentación del API del SSN__, he encontrado los campos que se deben mandar para obtener información en una tabla de html (nota: el servicio es __superlento__, Así que usaré pequeños rangos de fechas para no saturar la página del api)

In [2]:
for year in range(1990,2018):
    # Información en formato JSON para ser pedida por POST
    mData = {'ano1': str(year),
        'ano2': str(year),
        'dia1': '1',
        'dia2': '31',
        'estado': '',
        'filtrado': 'estado',
        'latmax': '33',
        'latmin': '13',
        'lonmax': '-85',
        'lonmin': '-118',
        'magnitud1': '0',
        'magnitud2': '9.5',
        'mes1': '1',
        'mes2': '12',
        'submit': 'BUSCAR'}
    r = requests.post("http://www2.ssn.unam.mx:8080/catalogo/",data = mData)
    with open('htmlb-'+str(year)+'.html','w') as file:
        file.write(r.text)
        print("Wrote backup file " + str(year))

Wrote backup file 1990
Wrote backup file 1991
Wrote backup file 1992
Wrote backup file 1993
Wrote backup file 1994
Wrote backup file 1995
Wrote backup file 1996
Wrote backup file 1997
Wrote backup file 1998
Wrote backup file 1999
Wrote backup file 2000
Wrote backup file 2001
Wrote backup file 2002
Wrote backup file 2003
Wrote backup file 2004
Wrote backup file 2005
Wrote backup file 2006
Wrote backup file 2007
Wrote backup file 2008
Wrote backup file 2009
Wrote backup file 2010
Wrote backup file 2011
Wrote backup file 2012
Wrote backup file 2013
Wrote backup file 2014
Wrote backup file 2015
Wrote backup file 2016
Wrote backup file 2017


Añadimos los sismos que van del 2018.

In [3]:
mData = {'ano1': '2018', 'ano2': '2018', 'dia1': '1', 'dia2': '17',
        'estado': '', 'filtrado': 'estado', 'latmax': '33', 'latmin': '13',
        'lonmax': '-85', 'lonmin': '-118', 'magnitud1': '0', 'magnitud2': '9.5',
        'mes1': '1', 'mes2': '2', 'submit': 'BUSCAR'}
r = requests.post("http://www2.ssn.unam.mx:8080/catalogo/",data = mData)
with open('htmlb-2018.html','w') as file:
    file.write(r.text)
    print("Wrote backup file 2018")

Wrote backup file 2018


Ahora creamos una _sopa_ con el resultado de nuestro request. Además de eso, de estos archivos vamos a crear una tabla con Pandas. Primero sacamos la tabla de datos del html, cuyos elementos se identifican por ser del tipo 'input'. El SSN pone 10 categorías, por así decirlo: archivo (0), evento (1), fecha (2), hora (3), latitud (4), longitud (5), profundidad (6), magnitud (7) y epicentro (8) (hay dos más, 'regresar' y 'titulo' pero son funciones de la página, no datos). Ahora, de manera no tan compleja, preparamos el DataFrame de Pandas (este código es LEEEEEENTOOOO):

In [4]:
resultingDF = pd.DataFrame(columns = ['Magnitud','Latitud','Longitud','Profundidad (km)', 'Fecha y Hora'])
for year in range(1990,2019)[::-1]:
    with open('htmlb-'+str(year)+'.html','r') as file:
        print("Creating soup and building DataFrame (" + str(year) + ")")
        soup = BeautifulSoup(file, 'html5lib')
        #con el parámetro {'name':'evento'} se puede pre-filtrar
        allTags = soup.find_all('input')

        #Ponemos en una lambda una funión para interpretar dos campos como un Timestamp
        dateparser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

        for i in range(0,len(allTags)//11):
            series = {'Magnitud':float(allTags[i*11+7]['value']),
                      'Latitud':allTags[i*11+4]['value'],
                      'Longitud':allTags[i*11+5]['value'],
                      'Profundidad (km)': allTags[i*11+6]['value'],
                     'Fecha y Hora': dateparser(allTags[i*11+2]['value'] + " " + allTags[i*11+3]['value'])}
            resultingDF = resultingDF.append(series, ignore_index=True)
        print("Done")

Creating soup and building DataFrame (2018)
Done
Creating soup and building DataFrame (2017)
Done
Creating soup and building DataFrame (2016)
Done
Creating soup and building DataFrame (2015)
Done
Creating soup and building DataFrame (2014)
Done
Creating soup and building DataFrame (2013)
Done
Creating soup and building DataFrame (2012)
Done
Creating soup and building DataFrame (2011)
Done
Creating soup and building DataFrame (2010)
Done
Creating soup and building DataFrame (2009)
Done
Creating soup and building DataFrame (2008)
Done
Creating soup and building DataFrame (2007)
Done
Creating soup and building DataFrame (2006)
Done
Creating soup and building DataFrame (2005)
Done
Creating soup and building DataFrame (2004)
Done
Creating soup and building DataFrame (2003)
Done
Creating soup and building DataFrame (2002)
Done
Creating soup and building DataFrame (2001)
Done
Creating soup and building DataFrame (2000)
Done
Creating soup and building DataFrame (1999)
Done
Creating soup and bu

Por último salvamos el Data Frame a un archivo csv.

In [5]:
resultingDF.to_csv('earthquakes.csv')